# MMCV

In [ ]:
from onnx import *
!pip install onnxruntime
import onnxruntime
import numpy as np
import torch
import os
#安装onnxrunime
wget https://github.com/microsoft/onnxruntime/releases/download/v1.8.1/onnxruntime-linux-x64-1.8.1.tgz
tar -zxvf onnxruntime-linux-x64-1.8.1.tgz
cd onnxruntime-linux-x64-1.8.1
os.environ['ONNXRUNTIME_DIR'] = '/kaggle/working/onnxruntime-linux-x64-1.8.1'
export ONNXRUNTIME_DIR=$(pwd)
export LD_LIBRARY_PATH=$ONNXRUNTIME_DIR/lib:$LD_LIBRARY_PATH

In [ ]:
#安装MMCV  有强烈的版本匹配问题,/cu110/torch1.7.0必须正确
#https://blog.csdn.net/yililjljl/article/details/124896663 查看torchvision,torch,cuda,mmcv版本匹配问题
#安装torchvision可以自动更新torch
!pip install torchvision==0.8.0  #这个会安装torch==1.7.0
#安装对应的mmcv包
!pip install mmcv-full==1.5.3 -f https://download.openmmlab.com/mmcv/dist/cu110/torch1.7.0/index.html
    
# Install mmdetection
#rm -rf mmdetection
#解决AttributeError: type object ‘Callable‘ has no attribute ‘_abc_registry‘
# !pip uninstall typing -y
git clone https://github.com/open-mmlab/mmdetection.git
cd mmdetection
pip install -v -e .
# install Pillow 7.0.0 back in order to avoid bug in colab
#!pip install Pillow==7.1.0
#!pip install tornado==5.1


In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

In [3]:
#保存路径
!mkdir checkpoints
#下载checkpoint

!wget -c http://download.openmmlab.com/mmdetection/v2.0/yolo/yolov3_d53_320_273e_coco/yolov3_d53_320_273e_coco-421362b6.pth \
      -O checkpoints/tmp.pth

子目录或文件 checkpoints 已经存在。
'wget' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [ ]:
#简单的调用执行
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

# Choose to use a config and initialize the detector
config = 'configs/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco.py'
# Setup a checkpoint file to load
checkpoint = 'checkpoints/tmp.pth'
# initialize the detector
model = init_detector(config, checkpoint, device='cuda:0')


In [ ]:
# Use the detector to do inference
img = 'demo/demo.jpg'
result = inference_detector(model, img)

# Let's plot the result
show_result_pyplot(model, img, result, score_thr=0.3)

In [ ]:
import copy
import os.path as osp
import mmcv
import numpy as np
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

     tips:
     register中module_dict是个字典，可以用来删除已经注册过的类

In [ ]:
import os
temp =os.listdir('/kaggle/input/face-mask-detection/annotations')
with open('ann.txt','w') as f:
    for item in temp:
        strs = item[:-4]
        f.write(strs+'\n')

In [ ]:
#自定义数据集
#XML格式的标签数据集制作
from mmdet.datasets import XMLDataset
import os.path as osp
import xml.etree.ElementTree as ET
from mmdet.datasets.builder import DATASETS

import mmcv
import numpy as np
from PIL import Image
@DATASETS.register_module()
class KittiTinyDataset(CustomDataset):

    CLASSES = ('Car', 'Pedestrian', 'Cyclist')#所有的类名称

    def load_annotations(self, ann_file):
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        # load image list from file
        image_list = mmcv.list_from_file(self.ann_file)
    
        data_infos = []#完成数据放入列表中，每个样本一个字典
        # convert annotations to middle format
        for image_id in image_list:
            filename = f'{self.img_prefix}/{image_id}.jpeg'
            image = mmcv.imread(filename)
            height, width = image.shape[:2]
    
            data_info = dict(filename=f'{image_id}.jpeg', width=width, height=height)
    
            # load annotations
            label_prefix = self.img_prefix.replace('image_2', 'label_2')
            lines = mmcv.list_from_file(osp.join(label_prefix, f'{image_id}.txt'))
    
            content = [line.strip().split(' ') for line in lines]
            bbox_names = [x[0] for x in content]
            bboxes = [[float(info) for info in x[4:8]] for x in content]
    
            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []
    
            # filter 'DontCare'
            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_labels.append(cat2label[bbox_name])
                    gt_bboxes.append(bbox)
                else:
                    gt_labels_ignore.append(-1)
                    gt_bboxes_ignore.append(bbox)

            data_anno = dict(
                bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                labels=np.array(gt_labels, dtype=np.long),
                bboxes_ignore=np.array(gt_bboxes_ignore,
                                       dtype=np.float32).reshape(-1, 4),
                labels_ignore=np.array(gt_labels_ignore, dtype=np.long))

            data_info.update(ann=data_anno)
            data_infos.append(data_info)

        return data_infos


In [ ]:
#XML格式的标签数据集制作
from mmdet.datasets import XMLDataset
import os.path as osp
import xml.etree.ElementTree as ET
from mmdet.datasets.builder import DATASETS

import mmcv
import numpy as np
from PIL import Image
@DATASETS.register_module()
class FaceMaskDetection(XMLDataset):
    
    #图片的类别
    CLASSES=('mask',)
    
    def __init__(self,**kwargs):
        super().__init__(img_subdir='images',ann_subdir='annotations',**kwargs)

    def load_annotations(self, ann_file):
        """Load annotation from XML style ann_file.
        Args:
            ann_file (str): Path of XML file.
        Returns:
            list[dict]: Annotation info from XML file.
        """
        
        data_infos = []
        img_ids = mmcv.list_from_file(ann_file)
        for img_id in img_ids:
            
            #修改路径
            filename = f'images/{img_id}.jpg'
            xml_path = osp.join(self.img_prefix, 'annotations',
                                f'{img_id}.xml')
            
            
            tree = ET.parse(xml_path)
            root = tree.getroot()
            size = root.find('size')
            if size is not None:
                width = int(size.find('width').text)
                height = int(size.find('height').text)
            else:
                
                #图片路径
                img_path = osp.join(self.img_prefix, 'images',
                                    '{}.jpg'.format(img_id))
                
                
                img = Image.open(img_path)
                width, height = img.size
            data_infos.append(
                dict(id=img_id, filename=filename, width=width, height=height))

        return data_infos

In [ ]:
from mmcv import Config
#模型
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py')

print(f'Config:\n{cfg.pretty_text}')

In [8]:
from mmdet.apis import set_random_seed
# Modify dataset type and path
#--------------------------------------data---------------------------------------
cfg.type = 'FaceMaskDetection'
cfg.data_root = '/kaggle/input/face-mask-detection/'

cfg.dataset_type = 'FaceMaskDetection'
cfg.data.test.type = 'FaceMaskDetectiont'
cfg.data.test.data_root = '/kaggle/input/face-mask-detection/'
cfg.data.test.ann_file = '/kaggle/working/mmdetection/ann.txt'
cfg.data.test.img_prefix = 'images'
cfg.data.test.proposal_file = None

cfg.data.train.type = 'FaceMaskDetection'
cfg.data.train.data_root = r'/kaggle/input/face-mask-detection/'
cfg.data.train.ann_file = '/kaggle/working/mmdetection/ann.txt'
cfg.data.train.img_prefix = ''
#cfg.data.train.dataset.proposal_file = None

cfg.data.val.type = 'FaceMaskDetection'
cfg.data.val.data_root = r'/kaggle/input/face-mask-detection/'
cfg.data.val.ann_file = '/kaggle/working/mmdetection/ann.txt'
cfg.data.val.img_prefix = ''
cfg.data.val.proposal_file = None
#----------------------------------------------------------------------

#----------------------------------model----------------------------------
cfg.model.bbox_head.num_classes=2

#------------------------------------------------------------------------------
# Set up working dir to save files and logs.
cfg.work_dir = './tutorial_exps'

norm_cfg = dict(type='BN', requires_grad=True)

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer.lr = 0.02/16
cfg.lr_config.warmup ='linear'
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'mAP'
# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 12
cfg.evaluation.iou_thr=[0.5,0.75,0.95]
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 12


cfg.model.pretrained=None
#重新训练
cfg.load_from =None
#继续训练
cfg.resume_from=None
#在模型内或者在参数外面，以后会修改
#cfg.train_cfg=cfg.model.pop('train_cfg')
#cfg.test_cfg=cfg.model.pop('test_cfg')

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'
cfg.workflow=[('train',2),('val',1)]
# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')

ModuleNotFoundError: No module named 'mmdet'

In [9]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector


# Build dataset
datasets = [build_dataset(cfg.data.train),build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
#当validation=True的时候才会进行评估
train_detector(model, datasets, cfg, distributed=False, validate=True)


ModuleNotFoundError: No module named 'mmdet'

# 部署

In [1]:
#保存模型
torch.save(model.state_dict(),'checkpoint')

#模型参数文件
modelpath='./configs/yolo/yolov3_d53_320_273e_coco.py'
checkpointpath ='checkpoints/tmp.pth'
picpath='demo/demo.jpg'
model.eval().to('cpu')

from mmdetection.tools.deployment.pytorch2onnx import pytorch2onnx
input_shape=(1,3,224,224)
pytorch2onnx(model, 
             picpath, 
             input_shape,
             normalize_cfg={'mean':[0.0,0.0,0.0],'std':[0.1,0.1,0.1]},
             verify=False,
             output_file='tmp.onnx',
            skip_postprocess=False,#直接导出模型，不做验证
            )

SyntaxError: keyword argument repeated (<ipython-input-1-5ffb5fc37b41>, line 24)

# 使用

In [ ]:
from mmcv.parallel import collate, scatter
from mmcv.runner import load_checkpoint
from mmdet.datasets.pipelines import Compose
from mmdet.core import get_classes
from mmdet.datasets import replace_ImageToTensor

#这种形式生成的图片会和模型一致
imgs='demo/demo.jpg'
imgs = [imgs]
cfg = model.cfg #需要模型参数
cfg.data.test.pipeline = replace_ImageToTensor(cfg.data.test.pipeline)
test_pipeline = Compose(cfg.data.test.pipeline)
datas = []
data = dict(img_info=dict(filename=img), img_prefix=None)
#要看resize的方式，有缩放因子scale_factor，Pad操作不会影响结果
data = test_pipeline(data)
datas.append(data)
data = collate(datas, samples_per_gpu=len(imgs))
data['img_metas'] = [img_metas.data[0] for img_metas in data['img_metas']]
data['img'] = [img.data[0] for img in data['img']]

    from mmdet.datasets.pipelines import Compose
    testpipe = cfg.data.test.pipeline
    pipe =Compose(testpipe)
    testdata= pipe({'img_info':{'filename':'demo/demo.jpg'},'img_prefix':None})

In [ ]:
import onnx
!pip install onnxruntime
import onnxruntime
import io
import numpy as np
from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx
from mmdet.core.export.model_wrappers import ONNXRuntimeDetector
from mmdet.core.export import build_model_from_cfg, preprocess_example_input

In [ ]:
#载入onnx格式的模型
output_file='tmp.onnx'
onnx_model = onnx.load(output_file)#直接将onnx的图载入
#检查一下
onnx.checker.check_model(onnx_model)

In [ ]:
#模型自带bbox2results函数
onnx_model = ONNXRuntimeDetector(output_file, model.CLASSES, 0)

#输入大小，
input_shape=(1,3,320,320)
#路径
input_img='demo/demo.jpg'
normalize_cfg = {'mean':[123.675, 116.28, 103.53],'std':[58.395, 57.12, 57.375]}

input_config = {
    'input_shape': input_shape,
    'input_path': input_img,
    'normalize_cfg': normalize_cfg
}
#如果是动态输入要修改大小
dynamic_export =False
if dynamic_export:
    # scale up to test dynamic shape
    h, w = [int((_ * 1.5) // 32 * 32) for _ in input_shape[2:]]
    h, w = min(1344, h), min(1344, w)
    input_config['input_shape'] = (1, 3, h, w)
    
#生成输入,这个是直接使用resize作为输入，使用时需要注意，会影响结果，scale_factor=1
one_img, one_meta = preprocess_example_input(input_config)
#one_img是直接不管尺度比例，resize到指定大小的图，scale_factor =1
img_list, img_meta_list = [one_img], [[one_meta]]


# pytorch 输出，可直接用于显示
'''
with torch.no_grad():

    pytorch_results = model(
        img_list,
        img_metas=img_meta_list,
        return_loss=False,
        rescale=True)[0]
        
        '''


In [ ]:
#onnxruntime
img_list = [_.cuda().contiguous() for _ in img_list]

#自动解码完成
onnx_results = onnx_model(
    img_list, img_metas=img_meta_list, return_loss=False)[0]

In [1]:
#可视化onnx的结果
show_img = one_meta['show_img']
score_thr = 0.3
out_file_ort, out_file_pt = None, None
#确定是str还是show_img，两个图片大小不一样的
onnx_model.show_result(
    show_img,
    onnx_results,
    score_thr=score_thr,
    show=True,
    win_name='ONNXRuntime',
    out_file=out_file_ort)

NameError: name 'one_meta' is not defined

In [1]:
#可视化pytorch的结果
with torch.no_grad():
    model.to('cuda:0')
    pytorch_results = model(
        img_list,
        img_metas=img_meta_list,
        return_loss=False,
        rescale=True)[0]
model.show_result(
    show_img,
    pytorch_results,
    score_thr=score_thr,
    show=True,
    win_name='PyTorch',
    out_file=out_file_pt)

NameError: name 'model' is not defined

# TENSORRT

   同onnxruntime是一样的
   
       !polygraphy surgeon sanitize /kaggle/working/mmdetection/tmp.onnx \
        --fold-constants \
        -o /kaggle/working/folded.onnx
        !polygraphy run /kaggle/working/folded.onnx \
        --trt  --onnxrt \
        --input-shapes input:[1,3,416,416] \
        --save-engine temp.plane

In [ ]:
imgs='demo/demo.jpg'
imgs = [imgs]
cfg = model.cfg
#需要模型的test对图片进行处理
cfg.data.test.pipeline = replace_ImageToTensor(cfg.data.test.pipeline)
test_pipeline = Compose(cfg.data.test.pipeline)
datas = []
data = dict(img_info=dict(filename=img), img_prefix=None)
data = test_pipeline(data)
datas.append(data)
data = collate(datas, samples_per_gpu=len(imgs))
#img_metas主要保存了scale_factor，将结果变成原图上的结果
data['img_metas'] = [img_metas.data[0] for img_metas in data['img_metas']]
data['img'] = [img.data[0] for img in data['img']]

In [ ]:
img_list, img_meta_list = data['img'], data['img_metas']

In [ ]:
from mmdet.core.export.model_wrappers import TensorRTDetector
# TensorRTDetector里面有一个engine的装饰器，如果需要自定义模型使用，需要
#修改TensorRTDetector 里面forward_test，修改其输入和输出的名字output_names = ['dets', 'labels']

In [ ]:
trt_model = TensorRTDetector('temp.plane', model.CLASSES, device_id=0)

In [ ]:
with torch.no_grad():
    trt_results = trt_model(
        img_list, img_metas=img_meta_list, return_loss=False)[0]

In [ ]:
#可视化结果
score_thr = 0.2
onnx_model.show_result(
    input_img,
    trt_results,
    score_thr=score_thr,
    show=True,
    win_name='TensorRT')

# 使用OPENVINO进行部署

In [ ]:
#安装 

#Step 3. Set Up and Update PIP to the Highest Version
!python -m pip install --upgrade pip

#Step 4. Install the Package [caffe,caffe2,onnx,pytorch,tensorflow2]
!pip install openvino-dev[onnx]
#pip install openvino-dev[tensorflow2,mxnet,caffe]

#如果出现缺少libpython3.7m.so.1.0，执行下面的
!apt-get install libpython3.7 --yes


In [ ]:
#测试
!mo --input_model='tmp.onnx'

In [ ]:
import time
from openvino.runtime import Core
import cv2 as cv

ie = Core()
      
model_xml = 'tmp.xml'
model_bin = 'tmp.bin'

net = ie.read_model(model= model_xml,
                      #weights = model_bin
                     )
compiled_model = ie.compile_model(model=net,device_name="CPU")
input_blob = next(iter(net.inputs))
#接受输出节点的字典,或者net.outputs是个列表
out_blob = next(iter(net.outputs))

n,c,h,w = net.inputs[input_blob].shape
print(n,c,h,w)

src = cv.imread('demo/demo.jpg')
image =cv.resize(src,(w,h))
#注意数值int8，进行标准化
image = np.float32(image)
image[:,:,]-= (np.float32(123.675),np.float32(116.28),np.float32(103.53))
image[:,:,]/=(np.float32(58.395),np.float32(57.12),np.float32(57.375))

input_data = np.expand_dims(np.transpose(image, (2, 0, 1)), 0).astype(np.float32)
input_data.shape

result = compiled_model([input_data])

In [ ]:
node= []
for item in result.keys():
    node.append(item)

In [ ]:
from mmdet.core import bbox2result
#可以将节点转化成列表的形式，再用字典取出
dets,labels = result[node[0]],result[node[1]]
bbox_results = bbox2result(dets,labels,80)

In [1]:
#可视化openvino的结果bbox_results
show_img = one_meta['show_img']
score_thr = 0.4
model.show_result(
    show_img,
    bbox_results,
    score_thr=score_thr,
    show=True,
    win_name='OPENVINO')

NameError: name 'one_meta' is not defined